# Sobol Sensitivity Indices for the BTD Model

## Setup packages.

In [1]:
require(data.table)
require(magrittr)
require(sensitivity)
require(SobolSequence)

require(ggplot2)

Loading required package: data.table
Loading required package: magrittr
Loading required package: sensitivity
Registered S3 method overwritten by 'sensitivity':
  method    from 
  print.src dplyr
Loading required package: SobolSequence
Loading required package: ggplot2


## Design experiment.

### Load input ranges.

In [2]:
z.ranges <- fread("input-ranges-si.tsv")
z.ranges %>% dim

[1] 32  9

In [3]:
z.ranges %>% summary

   Variable            Units               Type           Model Minimum   
 Length:32          Length:32          Length:32          Min.   :  0.00  
 Class :character   Class :character   Class :character   1st Qu.:  0.00  
 Mode  :character   Mode  :character   Mode  :character   Median :  0.01  
                                                          Mean   : 17.89  
                                                          3rd Qu.:  1.00  
                                                          Max.   :168.00  
 Model Maximum      Model Default       Sensitivity Minimum Sensitivity Maximum
 Length:32          Min.   :        0   Min.   :        0   Min.   :0.000e+00  
 Class :character   1st Qu.:        0   1st Qu.:        0   1st Qu.:3.000e+00  
 Mode  :character   Median :        1   Median :        0   Median :1.400e+01  
                    Mean   : 12547769   Mean   :  3764340   Mean   :5.865e+07  
                    3rd Qu.:     1664   3rd Qu.:      512   3rd Qu.:1.058e+

### Experiment based on Sobol sequences.

In [4]:
n <- 200
k <- dim(z.ranges)[1]

In [5]:
z.design <- sobolSequence.points(2 * k, count = n)
colnames(z.design) <- c(z.ranges$Variable, z.ranges$Variable)
z.design %>% dim

[1] 200  64

In [6]:
write.table(z.design, file = "design-si.tsv", row.names = FALSE, col.names = TRUE, sep = "\t", quote = FALSE)

### Relate the design to the model's variables.

In [7]:
z.ab <- z.design
for (i in 1:k) {
    for (j in c(i, k + i)) {
        z.ab[, j] <- z.ranges[i, `Sensitivity Minimum`] * (1 - z.ab[, j]) +
                     z.ranges[i, `Sensitivity Maximum`] *      z.ab[, j]
        if (z.ranges[i, `Type`] %in% c("Integer", "Boolean"))
            z.ab[, j] <- round(z.ab[, j])
    }
}
z.inputs <- rbind(
    data.table(Matrix = "A", Index = 0, z.ab[, 1:k    ]),
    data.table(Matrix = "B", Index = 0, z.ab[, 1:k + k])    
)
for (i in 1:k) {
    z.c <- z.ab[, 1:k + k]
    z.c[, i] <- z.ab[, i]
    z.inputs <- rbind(
        z.inputs,
        data.table(Matrix = "C", Index = i, z.c)
    )
}
z.inputs <- cbind(
    Run = 1:dim(z.inputs)[1],
    z.inputs
)
z.inputs %>% dim

[1] 6800   35

In [8]:
z.inputs %>% summary

      Run          Matrix              Index       aversion to NPV deviation
 Min.   :   1   Length:6800        Min.   : 0.00   Min.   :0.0600           
 1st Qu.:1701   Class :character   1st Qu.: 7.00   1st Qu.:0.7075           
 Median :3400   Mode  :character   Median :15.50   Median :1.3549           
 Mean   :3400                      Mean   :15.53   Mean   :1.3537           
 3rd Qu.:5100                      3rd Qu.:24.00   3rd Qu.:2.0024           
 Max.   :6800                      Max.   :32.00   Max.   :2.6498           
 base external investor ask rate bioproduct long term price
 Min.   : 2.400                  Min.   :1500              
 1st Qu.: 6.384                  1st Qu.:3368              
 Median :10.369                  Median :5236              
 Mean   :10.364                  Mean   :5224              
 3rd Qu.:14.353                  3rd Qu.:7103              
 Max.   :18.337                  Max.   :8971              
 bioproduct performance advantage commerc

In [9]:
write.table(z.inputs[, 1:3], file="indices-si.tsv", row.names = FALSE, col.names = TRUE, sep = "\t", quote = FALSE)

In [10]:
write.table(z.inputs[, -(2:3)], file="inputs-si.tsv", row.names = FALSE, col.names = TRUE, sep = "\t", quote = FALSE)

## Analyze results.

### Read files.

#### Read design.

In [ ]:
z.design <- fread("design-si.tsv")
z.design %>% dim

#### Read inputs.

In [ ]:
z.inputs <- fread("inputs-si.tsv")
z.inputs %>% dim

#### Read outputs.

In [ ]:
z.outputs <- fread("outputs-si.tsv")
z.outputs[Time == 2050] %>% dim

In [ ]:
z.outputs[Time == 2050] %>% summary

### Compute sensitivity indices.

#### Just use the final year.

In [ ]:
z.outputs.clean <- z.outputs[order(Run)][`Time` == 2050, c(-1, -2)] %>% as.matrix
z.outputs.clean %>% dim